## Clase 5: Preparación de modelos y Árboles de Decisión

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import tree

In [2]:
def read_data(archivo):
    df = pd.read_csv(archivo)
    df = df[['class',
            'cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment',
             'gill-spacing',
             'gill-size',
             'gill-color',
             'stalk-shape',
             'stalk-root',
             'stalk-surface-above-ring',
             'stalk-surface-below-ring',
             'stalk-color-above-ring',
             'stalk-color-below-ring',
             'veil-type',
             'veil-color',
             'ring-number',
             'ring-type',
             'spore-print-color']]
    return df

In [3]:
df = read_data("Data/hongos.csv")

In [4]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color
0,p,x,s,n,t,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k
1,e,x,s,y,t,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n
2,e,b,s,w,t,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n
3,p,x,y,w,t,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k
4,e,x,s,g,f,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n


In [5]:
def clean_data(df):
    n_nulls = df.isnull().sum() / df.shape[0]
    new_df = df[n_nulls[n_nulls < 0.5].index]  
    new_df = new_df.dropna()
    return new_df

In [6]:
def transform_data(df):
    df = df.apply(preprocessing.LabelEncoder().fit_transform)
    return df

In [7]:
def split_data(df, train, test):
    n = df.shape[0]
    n_train = int(np.round(n * train, 0))
    train = df.sample(n_train)
    test = df.loc[~df.index.isin(train.index)]
    return {'train': train, 'test': test}
    

In [9]:
data_raw = read_data('Data\hongos.csv')
cleaned_data = clean_data(data_raw)
trans_data = transform_data(cleaned_data)
split = split_data(trans_data, 0.7, 0.3)

In [10]:
split['train'].shape

(5687, 20)

### Árboles de Decisión

En la librería skit-learn, la función para el ajuste de árboles de decisión es ```DecisionTreeClassifier```, dentro del módulo ```tree```. Se puede usar simplemente entregando la matriz de features y el vector de labels.



In [11]:
datos = split

X = datos['train'][['cap-shape', 'cap-surface', 'cap-color']]
Y = datos['train'][['class']]

clf = tree.DecisionTreeClassifier(max_depth = 3)
clf = clf.fit(X, Y)

In [12]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

El modelo ya se encuentra ajustado, vamos a hacer una pequeña predicción

In [13]:
clf.predict(datos['test'][['cap-shape', 'cap-surface', 'cap-color']])

array([1, 0, 1, ..., 1, 1, 1])

Una forma rápida de evaluar es mediante la importancia de las características (basados en el gini)

In [14]:
clf.feature_importances_

array([0.2196326, 0.2091049, 0.5712625])

Más detalles del ajuste de árboles:

```DecisionTreeClassifier(criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1)```

Descripción de ellos

- **criterion:** El criterio para el ajuste del árbol, puede ser por gine para el CART o entropy para ID3
- **max_depth:** profundidad máxima del árbol, si es None sigue hasta que todas las hojas tienen una categoría o no quedan datos
- **min_samples_split:** Mínima cantidad de muestras para hacer una división.
- **min_samples_leaf:** Mínima cantidad de muestras para ser una hoja.
- **max_leaf_nodes:** Máxima cantidad de hojas


volvamos al árbol anterior.

In [15]:
X = datos['train'][['cap-shape', 'cap-surface', 'cap-color']]
Y = datos['train'][['class']]

clf = tree.DecisionTreeClassifier(max_depth = 3, criterion = 'entropy')
clf = clf.fit(X, Y)


In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
y_pred = clf.predict(datos['test'][['cap-shape', 'cap-surface', 'cap-color']])
y_real = datos['test'][['class']]

m = confusion_matrix(y_real, y_pred)

In [18]:
m

array([[ 573,  696],
       [ 154, 1014]], dtype=int64)

In [19]:
accuracy = (m[0, 0] + m[1,1]) / (m[0,0] + m[1,1] + m[0,1] + m[1,0])

In [20]:
accuracy

0.6512105047189167

Nuestra precisión es de 65,9% nada mal, hagamos un árbol más complejo.

In [21]:
clf2 = tree.DecisionTreeClassifier(max_depth = 10)
clf2 = clf2.fit(X, Y)

y_pred = clf2.predict(datos['test'][['cap-shape', 'cap-surface', 'cap-color']])
y_real = datos['test'][['class']]

m = confusion_matrix(y_real, y_pred)
accuracy = (m[0, 0] + m[1,1]) / (m[0,0] + m[1,1] + m[0,1] + m[1,0])
accuracy


0.7053754616331556

In [22]:
m

array([[926, 343],
       [375, 793]], dtype=int64)

Nuestro algoritmo ahora tiene aún más precisión, probemos agregando más variables.

In [23]:
X = datos['train'][[
            'cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment']]

clf3 = tree.DecisionTreeClassifier(max_depth = None)
clf3 = clf3.fit(X, Y)

y_pred = clf3.predict(datos['test'][[
            'cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment'
]])
y_real = datos['test'][['class']]

m = confusion_matrix(y_real, y_pred)
accuracy = (m[0, 0] + m[1,1]) / (m[0,0] + m[1,1] + m[0,1] + m[1,0])
accuracy


0.9076733688961839

Encontramos una precisión muy buena, veamos el árbol.

In [24]:
tree.export_graphviz(clf3,feature_names = ['cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment'], out_file = 'tree2.dot')   

por último podemos un poco el árbol reduciendo el número de hojas

In [57]:
X = datos['train'][[
            'cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment']]

clf4 = tree.DecisionTreeClassifier(max_depth = None, max_leaf_nodes = 10)
clf4 = clf4.fit(X, Y)

y_pred = clf4.predict(datos['test'][[
            'cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment'
]])
y_real = datos['test'][['class']]

m = confusion_matrix(y_real, y_pred)
accuracy = (m[0, 0] + m[1,1]) / (m[0,0] + m[1,1] + m[0,1] + m[1,0])
accuracy


0.8091916290521133

In [ ]:
tree.export_graphviz(clf4,feature_names = ['cap-shape',
             'cap-surface',
             'cap-color',
             'bruises',
             'gill-attachment'], out_file = 'tree3.dot') 

por mucho que haya bajado el accuracy, nos permite tener una mejor comprensión del proceso.